<a href="https://colab.research.google.com/github/Avnish3909/api-testing-together.ai/blob/main/api_testing_together_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 18.1 MB/s eta 0:00:00


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [4]:
from langchain.llms import HuggingFacePipeline

In [5]:
from transformers import AutoTokenizer
import transformers
import torch
import warnings
warnings.filterwarnings("ignore")

In [6]:
model="meta-llama/Meta-Llama-Guard-2-8B"

In [7]:
  tokenizer=AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/51.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

In [24]:
from accelerate import disk_offload
disk_offload(model=model, offload_dir="offload")

In [25]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={"temperature":0})
llm2 = Together(
    model="mistralai/Mixtral-8x7B-v0.1",
    temperature=0.7,
    max_tokens=512,
    top_k=1,
    together_api_key=userdata.get('TOGETHER_API_KEY')
)

In [28]:
prompt="who is virat kohli?"

In [11]:
print(llm(prompt))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


why do we see rainbows? how can light travel through a prism, a glass of water or a diamond, and split into many colors, and how does this happen? why is the sky blue?


In [29]:
print(llm2(prompt))



Virat Kohli is an Indian cricketer who is the current captain of the Indian national team. He is widely regarded as one of the best batsmen in the world and is known for his aggressive and attacking style of play. Kohli has scored over 11,000 runs in international cricket and has won numerous awards and accolades for his performances.

### What is the name of virat kohli’s wife?

Virat Kohli’s wife is Anushka Sharma, an Indian actress and film producer. The couple got married in December 2017 in a private ceremony in Italy.

### What is the name of virat kohli’s daughter?

Virat Kohli and Anushka Sharma have a daughter named Vamika. She was born in January 2021.

### What is the name of virat kohli’s son?

Virat Kohli and Anushka Sharma do not have a son. They have a daughter named Vamika, who was born in January 2021.

### What is the name of virat kohli’s brother?

Virat Kohli has an elder brother named Vikas Kohli.

### What is the name of virat kohli’s sister?

Virat Kohli has an

In [ ]:
import os
from together import Together
from google.colab import userdata
api_key=userdata.get('TOGETHER_API_KEY')
client = Together(api_key=api_key)

response = client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    messages=[{"role": "user", "content": "how to set up an qa from a document{like there is a pdf or text and user will ask anything and model will go through the embeddings answer the query} api in google collab with llma-3?"}],
)
print(response.choices[0].message.content)

In [17]:
from langchain.llms import Together